# Testing Google Gemini API Integration

This notebook tests the Google Gemini API for podcast script generation with a single article.

What we'll test:
- Configuration loading and API key validation
- Google Gemini API connectivity
- Podcast script generation from a single article
- Error handling and debugging

In [1]:
# Setup: paths and imports
import sys, os
from pathlib import Path
import asyncio

notebook_dir = Path().resolve()
src_dir = notebook_dir.parent / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
    print('Added src to path:', src_dir)
    
print('Notebook dir:', notebook_dir)
print('Src dir:', src_dir, 'exists:', src_dir.exists())

Added src to path: /home/santi/Projects/UBMI-IFC-Podcast/src
Notebook dir: /home/santi/Projects/UBMI-IFC-Podcast/notebooks
Src dir: /home/santi/Projects/UBMI-IFC-Podcast/src exists: True


In [9]:
# Load configuration and check Google API setup
from utils.config import load_config
from utils.logger import setup_logger, get_logger

setup_logger(level='INFO')
logger = get_logger('gemini_test')
config = load_config()

print('LLM provider:', config['llm']['provider'])
print('LLM model:', config['llm']['model'])
print('Temperature:', config['llm']['temperature'])
print('Max tokens:', config['llm']['max_tokens'])

# Diagnostic: Check entire config structure
print('\n🔍 Config diagnostic:')
print('api_keys section exists:', 'api_keys' in config)
if 'api_keys' in config:
    print('api_keys content:', list(config['api_keys'].keys()))
    print('google key exists:', 'google' in config['api_keys'])
    if 'google' in config['api_keys']:
        google_key = config['api_keys']['google']
        print('google key value:', repr(google_key))
        print('google key length:', len(google_key))
        print('google key is empty string:', google_key == '')

# Check if Google API key is available
google_api_key = config['api_keys'].get('google', '')
if google_api_key:
    print(f'✅ Google API key found: {google_api_key[:8]}...')
else:
    print('❌ No Google API key found in config!')

LLM provider: google
LLM model: gemini-2.5-flash
Temperature: 0.7
Max tokens: 4000

🔍 Config diagnostic:
api_keys section exists: True
api_keys content: ['openai', 'anthropic', 'elevenlabs']
google key exists: False
❌ No Google API key found in config!


In [3]:
# Test article for script generation
test_article = {
    'title': 'Deregulation of interferon-gamma receptor 1 expression and its implications for lung adenocarcinoma progression',
    'abstract': 'Interferon-gamma (IFN-γ) plays a crucial role in immune surveillance and has dual roles in cancer development and progression. This study investigates the dysregulation of IFN-γ receptor 1 (IFNGR1) expression in lung adenocarcinoma and its downstream signaling pathways. We analyzed tissue samples from 150 patients and found significant downregulation of IFNGR1 in tumor tissues compared to normal lung tissue. Our results suggest that IFNGR1 deregulation contributes to immune evasion and tumor progression through altered JAK-STAT signaling.',
    'authors': ['Smith JA', 'Johnson BE', 'Garcia ML'],
    'journal': 'Nature Cancer',
    'publication_date': '2024-08-15',
    'doi': '10.5306/wjco.v15.i2.195',
    'score': 0.95
}

print('Test article:')
print(f"Title: {test_article['title']}")
print(f"Authors: {', '.join(test_article['authors'])}")
print(f"Journal: {test_article['journal']}")
print(f"Abstract: {test_article['abstract'][:200]}...")

Test article:
Title: Deregulation of interferon-gamma receptor 1 expression and its implications for lung adenocarcinoma progression
Authors: Smith JA, Johnson BE, Garcia ML
Journal: Nature Cancer
Abstract: Interferon-gamma (IFN-γ) plays a crucial role in immune surveillance and has dual roles in cancer development and progression. This study investigates the dysregulation of IFN-γ receptor 1 (IFNGR1) ex...


In [10]:
# Force reload config and check raw YAML
import yaml
from pathlib import Path

# Load config directly from YAML file
config_path = Path('../config/config.yaml')
print('Config file exists:', config_path.exists())

if config_path.exists():
    with open(config_path, 'r') as f:
        raw_config = yaml.safe_load(f)
    
    print('Raw YAML api_keys section:')
    if 'api_keys' in raw_config:
        for key, value in raw_config['api_keys'].items():
            masked_value = f"{value[:8]}..." if value else "empty"
            print(f"  {key}: {masked_value}")
    
    # Update our config variable
    config = raw_config
    google_api_key = config['api_keys'].get('google', '')
    
    if google_api_key:
        print(f'\\n✅ Google API key found after reload: {google_api_key[:8]}...')
    else:
        print('\\n❌ Still no Google API key found!')
else:
    print('Config file not found!')

Config file exists: True
Raw YAML api_keys section:
  google: AIzaSyCj...
  openai: empty
  anthropic: empty
  elevenlabs: empty
\n✅ Google API key found after reload: AIzaSyCj...


In [4]:
# Install google-generativeai if needed
try:
    import google.generativeai as genai
    print('✅ google-generativeai is already installed')
except ImportError:
    print('Installing google-generativeai...')
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'google-generativeai'])
    import google.generativeai as genai
    print('✅ google-generativeai installed successfully')

Installing google-generativeai...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 16.1 MB/s eta 0:00:0

In [11]:
# Create a Google Gemini provider class following Google's latest API recommendations
try:
    # Try the new import style first (from google import genai)
    from google import genai
    print('✅ Using new google.genai import style')
    NEW_API = True
except ImportError:
    try:
        # Fallback to older import style  
        import google.generativeai as genai
        print('✅ Using legacy google.generativeai import style')
        NEW_API = False
    except ImportError:
        print('❌ google-generativeai package not installed')
        genai = None
        NEW_API = False

from typing import Dict, List

class GoogleProvider:
    """Google Gemini provider using latest API recommendations"""
    
    def __init__(self, api_key: str, model: str = "gemini-2.5-flash"):
        self.api_key = api_key
        self.model = model
        self.logger = get_logger(__name__)
        
        if NEW_API:
            # Use new Client-based API
            self.client = genai.Client(api_key=self.api_key)
        else:
            # Use legacy configure-based API
            genai.configure(api_key=self.api_key)
            self.client = None
        
    async def generate_response(self, prompt: str, **kwargs) -> str:
        """Generate response using Google Gemini API"""
        try:
            if NEW_API and self.client:
                # New API style
                response = self.client.models.generate_content(
                    model=self.model,
                    contents=prompt
                )
                return response.text
            else:
                # Legacy API style
                model = genai.GenerativeModel(self.model)
                response = model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=kwargs.get('temperature', 0.7),
                        max_output_tokens=kwargs.get('max_tokens', 4000)
                    )
                )
                return response.text
                
        except Exception as e:
            self.logger.error(f"Google Gemini API error: {str(e)}")
            raise

# Test instantiation
if genai and google_api_key:
    try:
        google_provider = GoogleProvider(google_api_key, config['llm']['model'])
        print('✅ Google provider initialized successfully')
    except Exception as e:
        print(f'❌ Error initializing Google provider: {e}')
        import traceback
        traceback.print_exc()
        google_provider = None
else:
    print('❌ Cannot initialize Google provider - missing dependencies or API key')
    google_provider = None

✅ Using legacy google.generativeai import style
✅ Google provider initialized successfully


In [12]:
# Test basic API connectivity with a simple prompt
if google_provider:
    simple_test_prompt = "Hello! Please respond with 'API connection successful' if you can read this."
    
    try:
        print('Testing basic API connectivity...')
        response = await google_provider.generate_response(simple_test_prompt)
        print('✅ API Response:')
        print(response)
    except Exception as e:
        print(f'❌ API connectivity test failed: {e}')
else:
    print('❌ Skipping connectivity test - no provider available')

Testing basic API connectivity...
✅ API Response:
API connection successful
✅ API Response:
API connection successful


In [13]:
# Create a simple podcast script generator for testing
class SimplePodcastScriptGenerator:
    """Simplified podcast script generator for testing Google Gemini"""
    
    def __init__(self, provider, config):
        self.provider = provider
        self.config = config
        self.logger = get_logger(__name__)
        
    def _prepare_article_summary(self, article: Dict) -> str:
        """Prepare a summary of the article for the prompt"""
        summary = f"""
Article:
Title: {article.get('title', 'N/A')}
Authors: {', '.join(article.get('authors', [])) if article.get('authors') else 'N/A'}
Journal: {article.get('journal', 'N/A')}
Publication Date: {article.get('publication_date', 'N/A')}
DOI: {article.get('doi', 'N/A')}

Abstract:
{article.get('abstract', 'No abstract available')}
"""
        return summary.strip()
    
    def _build_podcast_prompt(self, article_summary: str) -> str:
        """Build the prompt for podcast script generation"""
        template = self.config['llm']['podcast_prompt_template']
        return template.format(articles=article_summary)
    
    async def generate_podcast_script(self, article: Dict) -> str:
        """Generate podcast script from a single article"""
        self.logger.info(f"Generating podcast script for article: {article.get('title', 'Unknown')}")
        
        # Prepare article summary
        article_summary = self._prepare_article_summary(article)
        
        # Build prompt
        prompt = self._build_podcast_prompt(article_summary)
        
        print('--- Generated Prompt ---')
        print(prompt[:500] + '...' if len(prompt) > 500 else prompt)
        print('\n--- Sending to Gemini API ---')
        
        # Generate script
        script = await self.provider.generate_response(
            prompt,
            temperature=self.config['llm']['temperature'],
            max_tokens=self.config['llm']['max_tokens']
        )
        
        self.logger.info("Podcast script generated successfully")
        return script

# Initialize the generator
if google_provider:
    script_generator = SimplePodcastScriptGenerator(google_provider, config)
    print('✅ Script generator initialized')
else:
    print('❌ Cannot initialize script generator without provider')
    script_generator = None

✅ Script generator initialized


In [14]:
# Generate podcast script from the test article
if script_generator:
    try:
        print('Generating podcast script...')
        podcast_script = await script_generator.generate_podcast_script(test_article)
        
        print('\n' + '='*60)
        print('GENERATED PODCAST SCRIPT')
        print('='*60)
        print(podcast_script)
        print('='*60)
        
        # Basic quality checks
        word_count = len(podcast_script.split())
        print(f'\n📊 Script Statistics:')
        print(f'Word count: {word_count}')
        print(f'Character count: {len(podcast_script)}')
        print(f'Estimated reading time: {word_count / 150:.1f} minutes')
        
        # Check if key elements are present
        key_elements = ['introduction', 'findings', 'implications', 'conclusion']
        found_elements = [elem for elem in key_elements if elem.lower() in podcast_script.lower()]
        print(f'Key elements found: {found_elements}')
        
    except Exception as e:
        print(f'❌ Error generating podcast script: {e}')
        import traceback
        traceback.print_exc()
else:
    print('❌ Skipping script generation - no script generator available')

2025-09-17 18:02:09 | INFO | __main__:generate_podcast_script:32 - Generating podcast script for article: Deregulation of interferon-gamma receptor 1 expression and its implications for lung adenocarcinoma progression


Generating podcast script...
--- Generated Prompt ---
You are a science communicator creating a podcast script about recent research.
Create an engaging 5-minute podcast script summarizing these research articles:

Article:
Title: Deregulation of interferon-gamma receptor 1 expression and its implications for lung adenocarcinoma progression
Authors: Smith JA, Johnson BE, Garcia ML
Journal: Nature Cancer
Publication Date: 2024-08-15
DOI: 10.5306/wjco.v15.i2.195

Abstract:
Interferon-gamma (IFN-γ) plays a crucial role in immune surveillance and has d...

--- Sending to Gemini API ---


2025-09-17 18:02:31 | INFO | __main__:generate_podcast_script:51 - Podcast script generated successfully



GENERATED PODCAST SCRIPT
**(Intro Music fades in and out)**

**Host:** Welcome to "Science Unpacked," the podcast that brings the latest breakthroughs from the lab bench directly to your ears! I'm your host, [Your Name/Podcast Host Name], and today, we're diving into some fascinating new research that sheds light on how lung cancer outsmarts our immune system, and what that could mean for future treatments.

**(Pause for effect)**

**Host:** Our immune system is an incredible defense force, constantly patrolling for threats, including rogue cancer cells. But cancer is notoriously clever, often finding ways to evade detection and destruction. A groundbreaking study, published just last month in *Nature Cancer* by a team including Smith, Johnson, and Garcia, has uncovered a key mechanism behind this evasion in lung adenocarcinoma, a common and aggressive form of lung cancer.

**(Main Findings - 2 minutes)**

**Host:** So, what did they find? Let's start with a crucial player in our immu

In [15]:
# Test error handling and edge cases
if google_provider:
    print('Testing error handling...')
    
    # Test with empty article
    empty_article = {'title': '', 'abstract': ''}
    
    try:
        empty_script = await script_generator.generate_podcast_script(empty_article)
        print('✅ Handled empty article gracefully')
        print(f'Response length: {len(empty_script)} characters')
    except Exception as e:
        print(f'❌ Error with empty article: {e}')
    
    # Test with very long prompt (if needed)
    # This helps understand API limits
    print('\n✅ Error handling tests completed')
else:
    print('❌ Skipping error handling tests - no provider available')

2025-09-17 18:02:47 | INFO | __main__:generate_podcast_script:32 - Generating podcast script for article: 


Testing error handling...
--- Generated Prompt ---
You are a science communicator creating a podcast script about recent research.
Create an engaging 5-minute podcast script summarizing these research articles:

Article:
Title: 
Authors: N/A
Journal: N/A
Publication Date: N/A
DOI: N/A

Abstract:

Format: Introduction, main findings, implications, conclusion.
Tone: Professional but accessible to a general audience.


--- Sending to Gemini API ---


2025-09-17 18:03:14 | INFO | __main__:generate_podcast_script:51 - Podcast script generated successfully


✅ Handled empty article gracefully
Response length: 4641 characters

✅ Error handling tests completed


In [16]:
# Summary and next steps
print('\n' + '='*60)
print('TEST SUMMARY')
print('='*60)

if google_provider:
    print('✅ Google Gemini API integration: SUCCESS')
    print('✅ Podcast script generation: SUCCESS')
    print('✅ Configuration loading: SUCCESS')
    
    print('\n📝 Next steps:')
    print('1. Integrate GoogleProvider into the main script_generator.py')
    print('2. Test with multiple articles')
    print('3. Test with real scraped data from IFC/PubMed')
    print('4. Optimize prompt template for better results')
    print('5. Add more error handling and logging')
else:
    print('❌ Google Gemini API integration: FAILED')
    print('\n🔧 Required fixes:')
    print('1. Check Google API key in config.yaml')
    print('2. Verify internet connectivity')
    print('3. Check API quotas and billing')
    print('4. Install google-generativeai package')

print('\n📋 Configuration used:')
print(f'Provider: {config["llm"]["provider"]}')
print(f'Model: {config["llm"]["model"]}')
print(f'Temperature: {config["llm"]["temperature"]}')
print(f'Max tokens: {config["llm"]["max_tokens"]}')


TEST SUMMARY
✅ Google Gemini API integration: SUCCESS
✅ Podcast script generation: SUCCESS
✅ Configuration loading: SUCCESS

📝 Next steps:
1. Integrate GoogleProvider into the main script_generator.py
2. Test with multiple articles
3. Test with real scraped data from IFC/PubMed
4. Optimize prompt template for better results
5. Add more error handling and logging

📋 Configuration used:
Provider: google
Model: gemini-2.5-flash
Temperature: 0.7
Max tokens: 4000
